First, import the modules numpy, matplotlib, and os. They are used for numerical computation, plotting figures, and handling file operations, respectively. If these modules are not installed, you can install them using the following commands:  
  
pip install numpy  
pip install matplotlib  
pip install os  

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os

This function, calculate_avg_total_time, is used to compute the total time cost for each travel demand.  
Inputs:  
1. monorail_travel_time: a number, representing the in-vehicle monorail travel time.
2. train_processing_time_range: a list of two numbers, representing the processing-time range at high-speed rail stations.
3. flight_processing_time_range: a list of two numbers, representing the processing-time range at airports.
4. additional_train_stop: an array  of five numbers, indicating the number of additional stops in each high-speed rail segment.
5. return_time_range: a list of two numbers, representing the range of return times from the airport to downtown.

Outputs:  
1. avg_HSR_time: an array of five numbers, representing the average travel time for passengers using high-speed rail to each of the five cities.
2. avg_flight_time: an array of five numbers, representing the average travel time for passengers using a flight to each of the five cities.
    

In [12]:
def calculate_avg_total_time(monorail_travel_time,train_processing_time_range,flight_processing_time_range,additional_train_stop,return_time_range):
    #processing time range
    train_processing_time_min = train_processing_time_range[0]
    train_processing_time_max = train_processing_time_range[1]
    flight_processing_time_min = flight_processing_time_range[0]
    flight_processing_time_max = flight_processing_time_range[1]
    #return time range
    return_time_min = return_time_range[0]
    return_time_max = return_time_range[1]
    

    # statistic result
    traveler_number = {0:np.zeros(5),1:np.zeros(5)}
    statistic_result = {0:np.zeros(5),1:np.zeros(5)}
    
    # parameters
    # station ID
    HSR_station_index = 0
    Transit_monorail_station_index = 27
    # in-vehicle travel time list 0: high-speed rail, 1: airplane
    travel_time_list = {0:[97,150,190,230,300], 1:[60,70,80,90,130]}
    
    for i in range(100000):
        # Step 1 generate travel demand at a random station along the Yamanote line (0＝high-speed rail,1=flight)
        transport_type = np.random.randint(0, 2)
        start_station = np.random.randint(0, 30)
        # destination (Nagoya=0 Osaka=1 Okayama=2 Hiroshima=3 Hakata=4)
        destination_city = np.random.randint(5)
        
        # Step 2 access time (there are 30 stations along the Yamanote line)
        if transport_type==0:
            access_time = min((start_station-HSR_station_index)%30,(HSR_station_index-start_station)%30)*2
        elif transport_type==1:
            time_transit_to_monorail = np.random.randint(1, 5)
            time_using_monorail = monorail_travel_time
            access_time = min((start_station-Transit_monorail_station_index)%30,(Transit_monorail_station_index-start_station)%30)*2 + time_transit_to_monorail + time_using_monorail 
        
        # Step 3 processing time
        if transport_type==0:
            processing_time = np.random.randint(train_processing_time_min,train_processing_time_max) 
        elif transport_type==1:
            processing_time = np.random.randint(flight_processing_time_min,flight_processing_time_max)
        
        # Step 4 in-vehicle travel time
        if transport_type==0:
            travel_time = travel_time_list[transport_type][destination_city] + 6.5*np.sum(additional_train_stop[:destination_city+1])
        elif transport_type==1:
            travel_time = travel_time_list[transport_type][destination_city]
        
        # Step 5 return time (to downtown)
        if transport_type==0:
            return_time = 0
        elif transport_type==1:
            return_time = np.random.randint(return_time_min, return_time_max)
        
        # Step 6 total time
        total_time = access_time + processing_time + travel_time + return_time
        
        # Step 7 sum up all demands
        traveler_number[transport_type][destination_city]+=1
        statistic_result[transport_type][destination_city]+=total_time
    
    # calculate avg results
    avg_HSR_time = statistic_result[0]/traveler_number[0]
    avg_flight_time = statistic_result[1]/traveler_number[1]
    
    return avg_HSR_time, avg_flight_time

The function, draw_figure, is used to calculate the competitive range of high-speed rail and to generate a figure showing the time cost of high-speed rail and flights versus distance across different scenarios.  
Inputs:
1. avg_HSR_time: an array of five numbers, representing the average travel time for passengers using high-speed rail to each of the five cities.
2. avg_flight_time: an array of five numbers, representing the average travel time for passengers using a flight to each of the five cities.
3. case_ID: a string, representing the corresponding scenario.
4. path_save_picture_dir: the path where the generated figures will be saved.

Outputs: None

In [13]:
# Calculate competitive range and generate a figure
def generate_figure(avg_HSR_time,avg_flight_time,case_ID,path_save_picture_dir):
    # mileage (km)
    km_list = [366.0,552.6,732.9,894.2,1174.9]

    # calculate competitive range
    diff = avg_HSR_time - avg_flight_time
    x_cross = None
    for i in range(len(diff)-1):
        if diff[i] * diff[i+1] < 0:
            x1 = km_list[i]
            x2 = km_list[i+1]
            y1 = diff[i]
            y2 = diff[i+1]
            # find x to let diff = 0
            x_cross = round(x1 - y1 * (x2 - x1) / (y2 - y1),2)
            
            break

    # generate a figure
    fig = plt.figure(figsize=(8,6))
    ax_1 = fig.add_subplot(1,1,1)
    ax_1.plot(km_list,avg_HSR_time,c='r',label = 'High-Speed Rail')
    ax_1.plot(km_list,avg_flight_time,c='b',label = 'Flight')
    ax_1.set_xlabel('Distance (km)')
    ax_1.set_ylabel('Total time (min)')
    ax_1.set_title('Competitive range: '+str(x_cross)+' km')
    ax_1.legend()
    
    # save the figure
    plt.savefig(os.path.join(path_save_picture_dir,case_ID+'_comparison'), dpi=300,bbox_inches='tight')
    plt.close()

The following code creates a directory called figures to store the generated plots and analyzes the competitive range under different scenarios.

In [14]:
# create a directory to store figures
work = os.getcwd()
path_save_dir = os.path.join(work,'figures')
if not os.path.exists(path_save_dir):
    os.mkdir(path_save_dir)

# case 1: Current scenario
caseID = 'case1'
# calculate total time
case1_avg_HSR_time, case1_avg_flight_time = calculate_avg_total_time(25,[20,30],[60,90],np.array([0,0,0,0,0]),[40,60])
# generate a figure to show the results
generate_figure(case1_avg_HSR_time,case1_avg_flight_time,caseID,path_save_dir)

# case 2: With one more stop in each high-speed rail segment
caseID = 'case2'
# calculate total time
case2_avg_HSR_time, case2_avg_flight_time = calculate_avg_total_time(25,[20,30],[60,90],np.array([1,1,1,1,1]),[40,60])
# generate a figure to show the results
generate_figure(case2_avg_HSR_time,case2_avg_flight_time,caseID,path_save_dir)

# case 3: Shorter flight processing time
caseID = 'case3'
# calculate total time
case3_avg_HSR_time, case3_avg_flight_time = calculate_avg_total_time(25,[20,30],[40,60],np.array([0,0,0,0,0]),[40,60])
# generate a figure to show the results
generate_figure(case3_avg_HSR_time,case3_avg_flight_time,caseID,path_save_dir)

# case 4: Longer flight processing time
caseID = 'case4'
# calculate total time
case4_avg_HSR_time, case4_avg_flight_time = calculate_avg_total_time(25,[20,30],[80,120],np.array([0,0,0,0,0]),[40,60])
# generate a figure to show the results
generate_figure(case4_avg_HSR_time,case4_avg_flight_time,caseID,path_save_dir)

# case 5: Extreme case: one more stop in each high-speed rail segment and shorter flight processing time
caseID = 'case5'
# calculate total time
case5_avg_HSR_time, case5_avg_flight_time = calculate_avg_total_time(25,[20,30],[40,60],np.array([1,1,1,1,1]),[40,60])
# generate a figure to show the results
generate_figure(case5_avg_HSR_time,case5_avg_flight_time,caseID,path_save_dir)


